Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import joblib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import (
    train_test_split, TimeSeriesSplit, GridSearchCV
)
from sklearn.metrics import (
    mean_squared_error, r2_score,
    mean_absolute_error, make_scorer
)
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [2]:
# ─────────────────────── UTILIDADES / SCORERS ────────────────
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape_scorer = make_scorer(mape, greater_is_better=False)

Carga de datos

In [3]:
# 1) Carga y wide→long
df = pd.read_excel("datav2.xlsx", sheet_name=0) # Tienda 1
df_long = pd.wide_to_long(
    df, stubnames=["demanda","precio"],
    i="fecha", j="product_num", sep="", suffix=r"\d+"
).reset_index().rename(columns={
    "fecha":"ds","product_num":"unique_id",
    "demanda":"y","precio":"price"
})
df_long["unique_id"] = "prod_"+df_long["unique_id"].astype(str)
df_long["ds"] = pd.to_datetime(df_long["ds"])
df_long = df_long.sort_values(["unique_id","ds"])
df_largo = df_long.copy(deep=True)

df_largo.rename(columns={
    "unique_id":"Producto",
    "ds":"Semana",
    "y":"Demanda",
    "price":"Precio"
}, inplace=True)


In [4]:
print(df_largo.head(10))
print("...")
print(df_largo.tail(10))
print(50*"-")
print(len(df_largo)) # Deberia ser 2180 porque son 10 producto sy 218 semanas
print(df_largo.columns)

      Semana Producto  idx  Demanda  Precio
0 2021-01-01   prod_1    1     1950   28.79
1 2021-01-08   prod_1    2     2063   28.79
2 2021-01-15   prod_1    3     2392   35.99
3 2021-01-22   prod_1    4     3936   35.99
4 2021-01-29   prod_1    5     3616   35.99
5 2021-02-05   prod_1    6      636   28.79
6 2021-02-12   prod_1    7     1663   28.79
7 2021-02-19   prod_1    8     2433   28.79
8 2021-02-26   prod_1    9     2232   35.99
9 2021-03-05   prod_1   10     1299   35.99
...
         Semana Producto  idx  Demanda  Precio
1952 2024-12-27   prod_9  209     3200   39.99
1953 2025-01-03   prod_9  210     2962   31.99
1954 2025-01-10   prod_9  211     4255   31.99
1955 2025-01-17   prod_9  212     2003   39.99
1956 2025-01-24   prod_9  213     3081   39.99
1957 2025-01-31   prod_9  214     2910   39.99
1958 2025-02-07   prod_9  215     3922   31.99
1959 2025-02-14   prod_9  216     4944   31.99
1960 2025-02-21   prod_9  217     3340   31.99
1961 2025-02-28   prod_9  218     4706   3

In [5]:
# ═════════════════════ 2. PREPROCESAMIENTO ═══════════════════
df_largo['Semana'] = pd.to_datetime(df_largo['Semana'], errors='coerce')
df_largo['Semana_num'] = df_largo['Semana'].dt.isocalendar().week.astype(float)
df_largo['Mes'] = df_largo['Semana'].dt.month

le = LabelEncoder()
df_largo['Producto_cod'] = le.fit_transform(df_largo['Producto'])

df_largo['Precio']  = pd.to_numeric(df_largo['Precio'],  errors='coerce')
df_largo['Demanda'] = pd.to_numeric(df_largo['Demanda'], errors='coerce')
df_largo = df_largo.dropna(subset=['Precio', 'Demanda', 'Semana_num'])

# Variables de contexto
df_largo['Precio_promedio']  = df_largo.groupby('Producto')['Precio'].transform('mean')
df_largo['Demanda_promedio'] = df_largo.groupby('Producto')['Demanda'].transform('mean')
df_largo['Precio_relativo']  = df_largo['Precio'] / df_largo['Precio_promedio']

# Orden cronológico 
df_largo = df_largo.sort_values(['Producto', 'Semana'])
df_largo['Demanda_lag1'] = df_largo.groupby('Producto')['Demanda'].shift(1)
df_largo['Demanda_roll4'] = (
    df_largo.groupby('Producto')['Demanda'].shift(1).rolling(4).mean()
    .reset_index(level=0, drop=True)
)
df_largo = df_largo.dropna(subset=['Demanda_lag1', 'Demanda_roll4'])

# Estacionalidad 
df_largo['sin_season'] = np.sin(2 * np.pi * df_largo['Semana_num'] / 52)
df_largo['cos_season'] = np.cos(2 * np.pi * df_largo['Semana_num'] / 52)
df_largo['Demanda_log'] = np.log1p(df_largo['Demanda'])
df_largo = df_largo[df_largo['Demanda_log'].notna() & np.isfinite(df_largo['Demanda_log'])]

C:\Users\alexi\AppData\Roaming\Python\Python312\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [31]:
print(df_largo.head(10))
#print("...")
#print(df_largo.tail(10))

       Semana Producto  idx  Demanda  Precio  Semana_num  Mes  Producto_cod  \
5  2021-02-05   prod_1    6      636   28.79         5.0    2             0   
6  2021-02-12   prod_1    7     1663   28.79         6.0    2             0   
7  2021-02-19   prod_1    8     2433   28.79         7.0    2             0   
8  2021-02-26   prod_1    9     2232   35.99         8.0    2             0   
9  2021-03-05   prod_1   10     1299   35.99         9.0    3             0   
10 2021-03-12   prod_1   11     1868   39.59        10.0    3             0   
11 2021-03-19   prod_1   12     1460   43.19        11.0    3             0   
12 2021-03-26   prod_1   13     3194   43.19        12.0    3             0   
13 2021-04-02   prod_1   14     2171   43.19        13.0    4             0   
14 2021-04-09   prod_1   15     3833   43.19        14.0    4             0   

    Precio_promedio  Demanda_promedio  Precio_relativo  Demanda_lag1  \
5         38.378785       2379.397196         0.750154    

In [6]:
X = df_largo[[
    'Precio', 'Semana_num', 'Mes',
    'Precio_relativo', 'Demanda_promedio',
    'Demanda_lag1', 'Demanda_roll4',
    'sin_season', 'cos_season'
]]
y = df_largo['Demanda_log']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
print(X_test.head(10))
print(len(X_test))

      Precio  Semana_num  Mes  Precio_relativo  Demanda_promedio  \
74     43.19        22.0    6         1.128593       2383.174312   
2085   26.39        19.0    5         1.128614       7421.738532   
1373  107.99        13.0    4         1.128573       4828.201835   
1115   67.20        25.0    6         1.128559       2454.591743   
2034   26.39        20.0    5         1.128614       7421.738532   
1120   56.00        30.0    7         0.940466       2454.591743   
1079   38.39        51.0   12         1.128597       8653.986239   
728    79.19        22.0    6         1.128577       1647.188073   
563    47.99        23.0    6         1.128589       3945.550459   
1017   31.99        41.0   10         0.940448       8653.986239   

      Demanda_lag1  Demanda_roll4  sin_season    cos_season  
74          3721.0        2635.25    0.464723 -8.854560e-01  
2085        3753.0        3900.50    0.748511 -6.631227e-01  
1373        8433.0        2394.25    1.000000 -1.608123e-16  
111

In [9]:

modelo = XGBRegressor()
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
y_pred_lvl = np.expm1(y_pred)
y_test_lvl = np.expm1(y_test)
print(f"\n✅ RMSE: {mean_squared_error(y_test_lvl, y_pred_lvl) ** 0.5:.2f}")
print(f"✅ MAE:  {mean_absolute_error(y_test_lvl, y_pred_lvl):.2f}")
print(f"✅ R²:   {r2_score(y_test_lvl, y_pred_lvl):.4f}")
print(f"✅ MAPE: {mape(y_test_lvl, y_pred_lvl):.2f}")



✅ RMSE: 1705.00
✅ MAE:  1189.00
✅ R²:   0.6049
✅ MAPE: 30.35


Prueba con precios inflados:

In [10]:
X_test_fake = X_test.copy()
X_test_fake['Precio'] = 20000


print("Cabeza del escenario con mismos datos pero precios inflados:\n",X_test_fake.head(10))

# 2) Predicción “log” y paso a nivel (si tu modelo entrenó en log(demanda))
y_pred_fake_log = modelo.predict(X_test_fake)
y_pred_fake_lvl = np.expm1(y_pred_fake_log)

# 3) Cálculo de métricas contra la demanda real
mae  = mean_absolute_error(y_test_lvl, y_pred_fake_lvl)
rmse = np.sqrt(mean_squared_error(y_test_lvl, y_pred_fake_lvl))
r2   = r2_score(y_test_lvl, y_pred_fake_lvl)
mape = np.mean(np.abs((y_test_lvl - y_pred_fake_lvl) / y_test_lvl)) * 100

print("=== Métricas escenario price=20000 ===")
print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

Cabeza del escenario con mismos datos pero precios inflados:
       Precio  Semana_num  Mes  Precio_relativo  Demanda_promedio  \
74     20000        22.0    6         1.128593       2383.174312   
2085   20000        19.0    5         1.128614       7421.738532   
1373   20000        13.0    4         1.128573       4828.201835   
1115   20000        25.0    6         1.128559       2454.591743   
2034   20000        20.0    5         1.128614       7421.738532   
1120   20000        30.0    7         0.940466       2454.591743   
1079   20000        51.0   12         1.128597       8653.986239   
728    20000        22.0    6         1.128577       1647.188073   
563    20000        23.0    6         1.128589       3945.550459   
1017   20000        41.0   10         0.940448       8653.986239   

      Demanda_lag1  Demanda_roll4  sin_season    cos_season  
74          3721.0        2635.25    0.464723 -8.854560e-01  
2085        3753.0        3900.50    0.748511 -6.631227e-01  
137

In [13]:
print(y_pred_fake_lvl)
print(len(X_test_fake))
print(len(y_pred_fake_lvl))
print(type(y_pred_fake_lvl))

[ 1734.0413 11818.503   4018.6235  1591.2097 10878.72    2532.2656
  5566.7715  2083.4873  3666.9734  4970.6924  6415.63    2379.6091
  1560.3175  3253.6719  3202.6812  4039.4004  3156.9707  2022.9867
  2573.8513  5758.6626  3676.7566  5700.2314  3040.4136  6088.418
  4904.951   2598.5098  2666.0967  4541.818   9596.511   3600.295
  3815.4175  3591.9211  7413.748   5926.1104  2246.159   4262.3926
  1989.7557  1879.9956  2776.2192  3251.013   1708.7897  2955.113
  3505.1318  5681.076   4534.3853  3197.9766  2256.4004  5666.378
  2584.342   2514.8564  1708.0952  4568.4097  4709.099   5634.2993
  5969.841   1955.5481  1970.1453  1939.9315  5255.475   6711.1533
  4025.61    2728.3545  4496.1177  2549.8162  3467.5706  1963.2086
  1885.7034  2663.2659  3665.795   2881.9543  4251.1367  3895.5063
  2553.9185  1898.4711  4161.471   2829.908   7980.9014 12091.688
  2911.8416  4090.5767  3828.2761 10083.8545  1493.0967  2261.8923
  3965.9062  6150.974   3344.8997  8497.732   5330.3555  3898.7554


In [34]:
# ═════════════════════ 4. GRID SEARCH GLOBAL ════════════════
xgb_base = XGBRegressor()
param_grid = {
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth':     [3, 5, 7],
    'n_estimators':  [300, 600, 900, 1200],
    'subsample':     [0.7, 1],
    'colsample_bytree': [0.7, 1]
}


grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print("\n✅ Mejor combinación de hiperparámetros:")
print(grid_search.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits

✅ Mejor combinación de hiperparámetros:
{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 600, 'subsample': 0.7}


In [35]:
modelo_optimo = grid_search.best_estimator_
modelo_optimo.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=600,
             n_jobs=None, num_parallel_tree=None, ...)

Exportación de modelo entrenado:

In [ ]:
# Exportacion de modelo entrenado
joblib.dump(modelo_optimo, "xgb_modelo.pkl")

print("✅ Modelo XGBoost guardado en xgb_modelo.pkl")

✅ Modelo XGBoost guardado en xgb_modelo.pkl


In [36]:

y_pred_opt = modelo_optimo.predict(X_test)
y_pred_opt_lvl = np.expm1(y_pred_opt)

rmse = mean_squared_error(y_test_lvl, y_pred_opt_lvl) ** 0.5
mae  = mean_absolute_error(y_test_lvl, y_pred_opt_lvl)
r2   = r2_score(y_test_lvl, y_pred_opt_lvl)
mape_val = (abs(y_test_lvl - y_pred_opt_lvl) / y_test_lvl).mean() * 100


print(f"\n✅ RMSE optimizado: {rmse:.2f}")
print(f"✅ MAE optimizado:  {mae:.2f}")
print(f"✅ R² optimizado:   {r2:.4f}")
print(f"✅ MAPE:            {mape_val:.2f}%")


✅ RMSE optimizado: 1739.93
✅ MAE optimizado:  1219.59
✅ R² optimizado:   0.6156
✅ MAPE:            35.55%


In [37]:
# ════════════════ 5. MODELOS POR PRODUCTO ═══════════════════
print("\n📦 ENTRENANDO MODELO POR PRODUCTO...")
productos = df_largo['Producto'].unique()
mapes = []


for producto in productos:
    df_prod = df_largo[df_largo['Producto'] == producto].copy()
    df_prod = df_prod.sort_values('Semana')

    Xp = df_prod[[
        'Precio', 'Semana_num', 'Mes',
        'Precio_relativo', 'Demanda_promedio',
        'Demanda_lag1',       
        'Demanda_roll4',     
        'sin_season', 'cos_season'   
    ]]


    yp = df_prod['Demanda_log']
    

    
    mask = yp.notna() & np.isfinite(yp)
    Xp, yp = Xp.loc[mask], yp.loc[mask]

    cut = int(len(yp) * 0.8)              
    Xp_train, Xp_test = Xp.iloc[:cut], Xp.iloc[cut:]
    yp_train, yp_test = yp.iloc[:cut], yp.iloc[cut:]


    param_grid_prod = {
        'learning_rate': [0.01, 0.03, 0.05],
        'max_depth':     [3, 5, 7],
        'n_estimators':  [300, 600, 900, 1200],
        'subsample':     [0.7, 1],
        'colsample_bytree': [0.7, 1]
        }


    tscv = TimeSeriesSplit(n_splits=3)
    gs = GridSearchCV(
        estimator=XGBRegressor(
            objective='reg:squarederror',
            random_state=42,
            n_jobs=-1
        ),
        param_grid=param_grid_prod,
        scoring=mape_scorer,
        cv=tscv,
        verbose=0,
        n_jobs=-1
    )
    gs.fit(Xp_train, yp_train)

    best_model = gs.best_estimator_
    best_model.fit(Xp_train, yp_train)

    yp_pred_log = best_model.predict(Xp_test)
    yp_pred_lvl = np.expm1(yp_pred_log)
    yp_test_lvl = np.expm1(yp_test)
    mape_p = (abs(yp_test_lvl - yp_pred_lvl) / yp_test_lvl).mean() * 100

    mapes.append({
    "Producto":   producto,
    "MAPE_%":     round(mape_p, 2),
    "Profundidad": best_model.max_depth,
    "Árboles":     best_model.n_estimators
})

    print(f"📊 {producto} → MAPE: {mape_p:.2f}%")



📦 ENTRENANDO MODELO POR PRODUCTO...
📊 prod_1 → MAPE: 26.71%
📊 prod_10 → MAPE: 72.46%
📊 prod_2 → MAPE: 28.24%
📊 prod_3 → MAPE: 36.41%
📊 prod_4 → MAPE: 25.65%
📊 prod_5 → MAPE: 34.38%
📊 prod_6 → MAPE: 30.75%
📊 prod_7 → MAPE: 22.93%
📊 prod_8 → MAPE: 30.10%
📊 prod_9 → MAPE: 30.45%


In [ ]:
# 6. PRESENTACIÓN RESUMIDA
# ==========================================================
df_resumen = (pd.DataFrame(mapes)
                .sort_values("MAPE_%")
                .reset_index(drop=True))

print("\n📋  RESUMEN DE ERRORES POR PRODUCTO")
print(df_resumen.to_string(index=False))

mape_simple = df_resumen["MAPE_%"].mean()
print(f"\n✅ MAPE PROMEDIO (simple): {mape_simple:5.2f}%")

# gráfico de barras
plt.figure(figsize=(10, 4))
plt.bar(df_resumen["Producto"], df_resumen["MAPE_%"], color="teal")
plt.axhline(30, ls="--", color="gray", lw=0.8)   
plt.title("MAPE por Producto (ordenado)")
plt.ylabel("MAPE %")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
# ════════════════ GRÁFICO GLOBAL: PRED. vs REAL ═══════════════
plt.figure(figsize=(6, 6))
plt.scatter(y_test_lvl, y_pred_opt_lvl, alpha=0.5, color='teal')
plt.plot([y_test_lvl.min(), y_test_lvl.max()],
        [y_test_lvl.min(), y_test_lvl.max()],
        ls='--', color='gray', lw=1)
plt.title("Demanda real vs Demanda estimada (modelo global)")
plt.xlabel("Demanda real")
plt.ylabel("Demanda estimada")
plt.grid(True, ls=':', lw=0.5)
plt.tight_layout()
plt.show()